In [1]:
import torch.nn as nn
from torchvision import models
import torch
import torchvision.transforms as transforms
from PIL import Image
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import gc

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [3]:
import sys
import os

sys.path.append(os.path.abspath("../"))
from learning.Entity.CustomDataset import PokemonMoveDataset


dataset = PokemonMoveDataset('D:/tanaka/Documents/poke-move/data/my-dataset/dataset.txt', device)

In [4]:
from learning.Encoder.MoveEncoder import MoveEncoder
from learning.Encoder.PokemonEncoder import PokemonEncoder


pokemon_model = PokemonEncoder(device).to(device)
move_model = MoveEncoder(device).to(device)

poke_model_name = 'D:/tanaka/Documents/poke-move/learning/pokemon_model_2023-08-04.pth'
move_model_name = 'D:/tanaka/Documents/poke-move/learning/move_model_2023-08-04.pth'
pokemon_model.load_state_dict(torch.load(poke_model_name))
move_model.load_state_dict(torch.load(move_model_name))
pokemon_model.eval()
move_model.eval()

MoveEncoder(
  (bert): ConvBertModel(
    (embeddings): ConvBertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ConvBertEncoder(
      (layer): ModuleList(
        (0): ConvBertLayer(
          (attention): ConvBertAttention(
            (self): ConvBertSelfAttention(
              (query): Linear(in_features=768, out_features=384, bias=True)
              (key): Linear(in_features=768, out_features=384, bias=True)
              (value): Linear(in_features=768, out_features=384, bias=True)
              (key_conv_attn_layer): SeparableConv1D(
                (depthwise): Conv1d(768, 768, kernel_size=(9,), stride=(1,), padding=(4,), groups=768, bias=False)
                (pointwise): Conv1d(768, 384, kernel_size=(1,), stride=

In [5]:
pokemon_vectors = []
for i, pokemon in enumerate(dataset.pokemons):
    poke_vector = dataset.pokemon_to_vector(pokemon)
    for key in poke_vector.keys():
        poke_vector[key] = poke_vector[key].unsqueeze(0)
    v = pokemon_model(poke_vector).to('cpu')
    pokemon_vectors.append(v)
    del poke_vector
pokemon_vectors = torch.cat(pokemon_vectors, dim=0)




In [6]:
%load_ext memory_profiler

In [7]:
%%memit
from memory_profiler import profile

# プロファイリング対象の関数に`@profile`デコレータを追加
@profile
def your_function():
    move_vectors = []
    for i, move in enumerate(dataset.moves):
        move_vector = dataset.move_to_vector(move)
        for key in move_vector.keys():
            if key == 'description':
                move_vector[key] = [move_vector[key]]
                continue
            move_vector[key] = move_vector[key].unsqueeze(0)
        v = move_model(move_vector).to('cpu')
        move_vectors.append(v)
        del move_vector
        gc.collect()
    move_vectors = torch.cat(move_vectors, dim=0)
your_function()

In [ ]:
len(move_vectors)

31

In [ ]:
import random

move_rankings = []
pokemon_ids = [random.randint(1, 1010) for i in range(5)]
for pokemon_id in pokemon_ids:
    # 各ベクトルのユークリッド距離を計算
    distances = torch.norm(pokemon_vectors[pokemon_id] - move_vectors, dim=1)

    # 距離を用いてベクトルBの添え字を並び替え
    sorted_indices = torch.argsort(distances)
    move_rankings.append(sorted_indices)


In [ ]:
text = ''

for i, pokemon_id in enumerate(pokemon_ids):
    pokemon = dataset.pokemons[pokemon_id]
    text += f'選ばれたポケモン: {pokemon.name}\n'
    text += f'近い技 TOP100\n'
    for j, move_id in enumerate(move_rankings[i][:100]):
        move = dataset.moves[move_id]
        text += f'{j}: {move.name}'